<a href="https://colab.research.google.com/github/ciciwu/DNN-playground/blob/master/nn_factorizaiton_machine_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch.nn as nn
import torch

In [2]:
embedding=nn.Embedding(4,2)


In [4]:
x=torch.tensor([3,2,1,0])

In [8]:
embedding(x)

tensor([[ 0.5803,  0.1145],
        [-0.9930, -0.0902],
        [ 0.2775,  1.0527],
        [ 1.1221,  2.0111]], grad_fn=<EmbeddingBackward0>)

In [12]:
 torch.pow(embedding(x), 2)

tensor([[0.3368, 0.0131],
        [0.9860, 0.0081],
        [0.0770, 1.1082],
        [1.2592, 4.0447]], grad_fn=<PowBackward0>)

In [10]:
torch.pow(embedding(x), 2).sum(dim=1, keepdim=True)

tensor([[0.3499],
        [0.9941],
        [1.1852],
        [5.3039]], grad_fn=<SumBackward1>)

In [5]:
pairwise_interactions = torch.sum(
    torch.pow(embedding(x), 2) -
    torch.pow(embedding(x), 2).sum(dim=1, keepdim=True),
    dim=1, keepdim=True)

In [11]:
pairwise_interactions

tensor([[-0.3499],
        [-0.9941],
        [-1.1852],
        [-5.3039]], grad_fn=<SumBackward1>)

In [ ]:
class FactorizationMachine(nn.Module):
    def __init__(self, num_features, embedding_dim):
        super(FactorizationMachine, self).__init__()

        # Linear term
        self.linear = nn.Linear(num_features, 1)

        # Pairwise interactions (factorization) term
        self.embedding = nn.Embedding(num_features, embedding_dim)

    def forward(self, x):
        # Linear term: sum of biases and weights for each feature
        linear_term = self.linear(x).sum(dim=1, keepdim=True)

        # Pairwise interactions term: sum of element-wise products of embeddings
        pairwise_interactions = torch.sum(torch.pow(self.embedding(x), 2) - torch.pow(self.embedding(x), 2).sum(dim=1, keepdim=True), dim=1, keepdim=True) * 0.5

        # Final prediction is the sum of linear and pairwise terms
        prediction = linear_term + pairwise_interactions

        return prediction